In [1]:
import datetime
import numpy as np
import pandas as pd
from pandas import Series
from itertools import groupby
import matplotlib.pyplot as plt

%matplotlib inline  

In [2]:
df = pd.read_csv('../../data/train.csv')
ids = df['KPI ID'].unique()

In [4]:
df.head()

,timestamp,value,label,KPI ID
0,1493568000,1.901639,0.0,02e99bd4f6cfb33f
1,1493568060,1.786885,0.0,02e99bd4f6cfb33f
2,1493568120,2.000000,0.0,02e99bd4f6cfb33f
3,1493568180,1.885246,0.0,02e99bd4f6cfb33f
4,1493568240,1.819672,0.0,02e99bd4f6cfb33f


In [5]:
df['label'].value_counts()

0.0    255677
1.0     10780
Name: label, dtype: int64

In [6]:
df['value'].describe()

count    266457.000000
mean         16.305243
std          16.933378
min           0.689655
25%           2.062500
50%           3.333333
75%          29.333334
max          91.307693
Name: value, dtype: float64

In [15]:
# upsampling
dfs = []
for _id in ids:
    df_tmp = df[df['KPI ID'] == _id]
    df_tmp['datetime'] = pd.to_datetime(df_tmp['timestamp'], unit='s')
    df_tmp = df_tmp.set_index('datetime')
    df_tmp = df_tmp.sort_index()
    df_tmp = df_tmp.resample('1T').ffill()
    
    df_tmp['id'] = df_tmp['KPI ID'] + '-' + df_tmp.index.astype('str') 
    df_tmp = df_tmp.set_index('id')
    
    dfs.append(df_tmp)
    print('Done with {_id}')
    
df_sampled = pd.concat(dfs)

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}
Done with {_id}


In [16]:
df_sampled[df_sampled['KPI ID'] == ids[1]].describe()
df_sampled.describe()

,timestamp,value,label
count,2.788680e+06,2.788680e+06,2.788680e+06
mean,1.493865e+09,9.792458e+07,2.108524e-02
std,7.829410e+06,3.057543e+08,1.436686e-01
min,1.467302e+09,-4.546069e+00,0.000000e+00
25%,1.493635e+09,2.000000e+00,0.000000e+00
50%,1.495959e+09,1.144000e+03,0.000000e+00
75%,1.498329e+09,2.372000e+03,0.000000e+00
max,1.502447e+09,1.608813e+09,1.000000e+00


In [13]:
# normalization
for _id in ids:
    df_tmp = df_sampled[df_sampled['KPI ID'] == _id]['value']
    df_sampled.loc[df_sampled['KPI ID'] == _id, 'value'] = (df_tmp - df_tmp.mean()) / df_tmp.std()

In [14]:
df_sampled[df_sampled['KPI ID'] == ids[2]].describe()

,timestamp,value,label
count,9.409000e+03,9.409000e+03,9409.000000
mean,1.493850e+09,2.103866e-15,0.009459
std,1.629745e+05,1.000000e+00,0.096802
min,1.493568e+09,-1.822134e+00,0.000000
25%,1.493709e+09,-8.701940e-01,0.000000
50%,1.493850e+09,-6.470658e-02,0.000000
75%,1.493991e+09,7.407809e-01,0.000000
max,1.494132e+09,8.209846e+00,1.000000


In [15]:
df_sampled.to_csv('../../data/train_sampled_normalized.csv')

In [3]:
# Preprocessing using KPI ID wise concat and linear interpolation for values (Variant 2)

df['timestamp'] = pd.to_datetime(df['timestamp'], unit = 's')
df = df.set_index('timestamp')

In [4]:
df_upsampled = pd.DataFrame()
for _id in ids:
    df_tmp_value = df[df['KPI ID'] == _id]['value']
    df_tmp_value_sampled = df_tmp_value.resample('1T').interpolate()
    df_tmp_label = df[df['KPI ID'] == _id]['label']
    df_tmp_label_sampled = df_tmp_label.resample('1T').ffill()
    df_tmp_label_sampled = df_tmp_label_sampled.to_frame('label')
    df_tmp_label_sampled.insert(1, 'KPI ID', _id)

    df_tmp_final = pd.concat([df_tmp_value_sampled, df_tmp_label_sampled], axis = 1, join='inner')
    df_tmp_final['id'] = df_tmp_final['KPI ID'] + '-' + df_tmp_final.index.astype('str') 
    df_upsampled = pd.concat([df_upsampled, df_tmp_final])

df_upsampled = df_upsampled.set_index('id')

In [5]:
# normalization
for _id in ids:
    df_tmp = df_upsampled[df_upsampled['KPI ID'] == _id]['value']
    df_upsampled.loc[df_upsampled['KPI ID'] == _id, 'value'] = (df_tmp - df_tmp.mean()) / df_tmp.std()

In [7]:
df_upsampled.to_csv('../../data/train_norm_KPIconcat_interpolated.csv')

In [8]:
df_upsampled.describe()

,value,label
count,2.788680e+06,2.788680e+06
mean,-1.349294e-15,2.108524e-02
std,9.999955e-01,1.436686e-01
min,-5.074012e+00,0.000000e+00
25%,-6.386425e-01,0.000000e+00
50%,2.659609e-02,0.000000e+00
75%,6.515980e-01,0.000000e+00
max,1.421557e+02,1.000000e+00
